In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import jinja2
import cv2
import logging

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow import keras
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
def identity_block(x, filter):
    # copy tensor to variable called x_skip
    x_skip = x
    # Layer 1
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation('relu')(x)
    # Layer 2
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    # Add Residue
    x = tf.keras.layers.Add()([x, x_skip])     
    x = tf.keras.layers.Activation('relu')(x)
    return x

In [ ]:
def convolutional_block(x, filter):
    # copy tensor to variable called x_skip
    x_skip = x
    # Layer 1
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same', strides = (2,2))(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation('relu')(x)
    # Layer 2
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    # Processing Residue with conv(1,1)
    x_skip = tf.keras.layers.Conv2D(filter, (1,1), strides = (2,2))(x_skip)
    # Add Residue
    x = tf.keras.layers.Add()([x, x_skip])     
    x = tf.keras.layers.Activation('relu')(x)
    return x

In [ ]:
def ResNet34(shape = (32, 32, 3), classes = 1):
    # Step 1 (Setup Input Layer)
    x_input = tf.keras.layers.Input(shape)
    x = tf.keras.layers.ZeroPadding2D((3, 3))(x_input)
    # Step 2 (Initial Conv layer along with maxPool)
    x = tf.keras.layers.Conv2D(16, kernel_size=7, strides=2, padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same')(x)
    # Define size of sub-blocks and initial filter size
    block_layers = [3, 4, 6, 3]
    filter_size = 16
    # Step 3 Add the Resnet Blocks
    for i in range(4):
        if i == 0:
            # For sub-block 1 Residual/Convolutional block not needed
            for j in range(block_layers[i]):
                x = identity_block(x, filter_size)
        else:
            # One Residual/Convolutional Block followed by Identity blocks
            # The filter size will go on increasing by a factor of 2
            filter_size = filter_size*2
            x = convolutional_block(x, filter_size)
            for j in range(block_layers[i] - 1):
                x = identity_block(x, filter_size)
    # Step 4 End Dense Network
    x = tf.keras.layers.AveragePooling2D((2,2), padding = 'same')(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(512, activation = 'relu')(x)
    x = tf.keras.layers.Dense(classes, activation = 'softmax')(x)
    model = tf.keras.models.Model(inputs = x_input, outputs = x, name = "ResNet34")
    model.compile(loss=compile_loss,  optimizer=lr, metrics=['accuracy'])
    return model

In [ ]:
data_dir_train = 'Dataset/train/'
data_dir_test = 'Dataset/test/'

img_size = 128
input_shape = (img_size, img_size, 3)
target_size =(img_size,img_size)
batch_size = 128
num_classes = 29
classes = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l','m', 'n', 'o', 'p', 'q', 'r', 
           's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'del', 'nothing', 'space']
           
test_num = 0

compile_optimizer = "adam"
compile_loss = "categorical_crossentropy"
model_type = "ResNet34"
learning_rate= 0.001
lr = tf.keras.optimizers.Adam(learning_rate=learning_rate)

csv_path = f'Results/Dataset_{model_type}_{batch_size}_{img_size}_Test_{test_num}_Learning_rate{learning_rate}.csv'
hist_path = f'Results/History/Dataset_{model_type}_{batch_size}_{img_size}_Test_{test_num}_Learning_rate{learning_rate}_history.csv'
save_model_path = f"Models/Dataset_{model_type}_{batch_size}_{img_size}_Test_{test_num}_Learning_rate{learning_rate}.h5"

In [ ]:
training_data = ImageDataGenerator(
        rescale = 1./255,
        rotation_range = 25,
        width_shift_range = 0.25,
        height_shift_range = 0.25,
        shear_range = 0.5,
        zoom_range = 0.25,
        brightness_range= [0.6,0.9],
        vertical_flip = True,
        fill_mode = 'nearest',
        validation_split = 0.2,
        )

test_data = ImageDataGenerator(
        rescale = 1./255,
        )

In [ ]:
train_ds = training_data.flow_from_directory(
    directory=data_dir_train,
    shuffle = True,
    seed = 42,
    color_mode="rgb",
    class_mode = 'categorical',
    target_size=target_size,
    batch_size=batch_size,
    subset="training",
   )

validation_ds = training_data.flow_from_directory(
    directory=data_dir_train,
    shuffle = True,
    seed = 42,
    class_mode = 'categorical',
    color_mode="rgb",
    target_size=target_size,
    batch_size=batch_size,
    subset="validation",
    )

test_ds = test_data.flow_from_directory(
    directory=data_dir_test,
    target_size=target_size,
    color_mode="rgb",
    shuffle = False,
    )

In [ ]:
model = ResNet34(input_shape, num_classes)
#model.load_weights('Models/DatasetV2_ResNet34_32_200_Test_1.h5')
#model.load_weights(save_model_path)
#model.summary()

In [ ]:
checkpoint = ModelCheckpoint(save_model_path, monitor='val_accuracy',
                             save_best_only=True, save_weights_only=True, mode='auto')

early = EarlyStopping(monitor='val_accuracy', patience=3, verbose=1)

history = model.fit(train_ds, epochs=50,  callbacks=[checkpoint,early], validation_data = validation_ds)

In [ ]:
model.load_weights(save_model_path)
#history = pd.read_csv(hist_path)

df = pd.DataFrame(history.history)
df.plot(figsize=(10,8))
df.to_csv(hist_path)

val_data = model.evaluate(validation_ds)

In [ ]:
history = pd.read_csv(hist_path)
df = pd.DataFrame(history[['accuracy', 'val_accuracy']])
df.plot(figsize=(10,8))

In [ ]:
Y_pred = model.predict(validation_ds)
print(Y_pred.shape)

In [ ]:
eq = []
for x in Y_pred:
    eq.append(np.argmax(x))


In [ ]:
cf_matrix = confusion_matrix(validation_ds.classes,eq)

In [ ]:


plt.figure(figsize = (50,50))

sns.heatmap(cf_matrix/np.sum(cf_matrix), annot=True, 
            fmt='.2%', cmap='Blues',
            annot_kws={
                'fontsize': 18,
                'fontweight': 'bold',
                'fontfamily': 'serif'
            },
            )
plt.xticks(range(0,29), classes)
plt.yticks(range(0,29), classes)
plt.tick_params(labelsize=40)

In [ ]:
pred = model.predict(test_ds, steps = len(test_ds), verbose=1)
cl = np.round(pred)
classes_prediction = []
for prediction in cl:
    classes_prediction.append(classes[np.argmax(prediction)])

filenames=test_ds.filenames
results=pd.DataFrame({"file":filenames,"prediction":pred[:,0], "class":classes_prediction})

In [ ]:
results.to_csv(csv_path)
#results.style

In [ ]:
import cv2 
rez = pd.read_csv(csv_path)
plt.figure(figsize=(60,60))
for i in range(28):
    img_path = str(rez['file'][i])
    image_path = data_dir_test + 'test/' + img_path[5:]
    #print(image_path)
    class_pred = str(rez['class'][i])
    img = cv2.imread(image_path)
    plt.subplot(7,8, i+1)
    plt.imshow(img)
    plt.title(class_pred, fontsize=20)
